In [0]:
%sql
SELECT * FROM binaryFile.`/Volumes/gizmobox/landing/operational_data/membership/*/*.png`

In [0]:
%sql
-- COMMAND ----------

CREATE OR REPLACE VIEW gizmobox.bronze.v_memberships
AS
SELECT * FROM binaryFile.`/Volumes/gizmobox/landing/operational_data/membership/*/*.png`


In [0]:
%sql
SELECT * FROM gizmobox.bronze.v_memberships;

In [0]:
%sql
SELECT regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
       content AS membership_card
  FROM gizmobox.bronze.v_memberships;

In [0]:
%sql
CREATE TABLE gizmobox.silver.memberships
AS
SELECT regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
       content AS membership_card
  FROM gizmobox.bronze.v_memberships;

-- COMMAND ----------

SELECT * FROM gizmobox.silver.memberships;

In [0]:
%sql
CREATE OR REPLACE VIEW gizmobox.silver.v_memberships_with_icon_url AS
SELECT
    customer_id,
    concat('data:image/png;base64,', base64(membership_card)) AS membership_card_icon_url
FROM gizmobox.silver.memberships;

SELECT * FROM gizmobox.silver.v_memberships_with_icon_url LIMIT 10;

In [0]:
%sql
CREATE TABLE gizmobox.gold.customer_payments AS
SELECT
    p.payment_id,
    p.order_id,
    p.payment_date,
    p.payment_time,
    p.payment_status,
    p.payment_method,
    c.customer_id,
    c.customer_name,
    c.email,
    c.created_timestamp,
    c.date_of_birth,
    c.member_since,
    c.telephone
FROM
    gizmobox.silver.payments p -- Alias the payments table as 'p'
JOIN
    gizmobox.silver.customers c ON p.customer_id = c.customer_id; -- Join based on customer_id